In [4]:
import re
import pandas as pd
import numpy as np
import PyPDF2
import nltk

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/user/nltk_data...


True

In [6]:
from nltk.corpus import wordnet


text embeddings
responsabitlity -> get a column of text data points and generate a vector for each of data points
Malaria -> [0.2, 0.992, 0.000...]

In [10]:
#import gensim

synonyms = []
antonyms = []

for syn in wordnet.synsets("stomach"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
             antonyms.append(l.antonyms()[0].name())

print(set(synonyms))
print(set(antonyms))

{'brook', 'tummy', 'abdomen', 'belly', 'endure', 'digest', 'venter', 'bear', 'stand', 'abide', 'put_up', 'stick_out', 'breadbasket', 'tum', 'stomach', 'support', 'suffer', 'tolerate'}
set()


clustering
group the data points
[0.2, 0.992, 0.000...] -> 1
[0.2, 0.992, 0.100...] -> 1
...
[0.9, 0., 0.31...] -> 21

Distance matrix

### NaturDoc - TL BL WT 22-23

# Data classification:

Diseases and symptoms

Kaggle etc notebooks on disease prediction based on symptoms

Inside Duke dataset right now: Activity column contains both diseases and symptoms without differentiating between them

-> ML: disease prediction based on simple symptoms mentioned, so that the app could also propose herbs that treat the underlying disease instead of just treating the symptoms

### Analysis:

Embeddings to generate data -> use that to train our classification models

Embedding of herbs - proximity/clustering of herbs based on their similarity (_what is "similar"_?)

Output: groups of herbs, have similar family/genus (?) but do they also have similar uses?

ML approach:
1. find lists of symptoms/values that are more human friendly
2. generate synonyms/word mapping associated with these symptoms
    * group these into different categories based on "linguistic distance"?
3. compare to activities
    * maybe first also generate synonyms/word mapping of terms in activities
4. an accuracy score of match between symptom and synonyms compared to activity

* is this considered ML?
* supervised/unsupervised? We don't exactly have access to labelled data, do we?

Could we create our own labelled data by associating a symptom with its synonyms? Maybe these features could not just be synonyms but based on semantic proximity? Get this "semantic proximity" from wordmapping libraries/unsupervised ML scripts?

| label | symptom | synonyms (including the symptom itself?) | related terms | least related term (largest distance within set of data) |
| --- | --- | --- | --- | --- |
| 1 | stomach ache | stomach ache, stomach pain, abdominal pain,... | stomach, abdomen, stomach cancer, intestine,... | sore throat, ... |

f: X -> y

What do we want here?

We want the input of a "symptom" to match the "medicinal use" of a given herb (initially referred to as "ACTIVITY" in the Duke dataset).

Symptoms are a pre-defined list of terms that users can choose from (autocomplete).

E.g. a user chooses "stomach ache" which should match relevant terms in the Duke dataset, such as "Ache(Stomach)", "Abdomen" and so on. 

__We want to train a model to be given a symptom, recognise related terms from "medicinal uses".__

f: X -> y

f: Symptom -> Activities 

f: "stomach ache" -> "Ache(Stomach)", "Abdomen", "Stomach" 

-> more distant matches: intestine, polyps, cancers etc. in the stomach

Or cluster data? Unsupervised. 

Cluster symptoms, synonyms and cluster activity 

CLUSTERS:

Use word embeddings for symptoms and activities each

cluster results of embeddings

compare the clusters, group proximity

In [46]:
duke = pd.read_csv("../data/DUKE/ETHNOBOT.csv")
print(duke.ACTIVITY.unique().tolist()[::40])

['Abcess', 'Alcoholism', 'Antacid', 'Antidote(Cicuta)', 'Antidote(Stingray)', 'Antismoking', 'Arthritis', 'Bilharzia', 'Bloodshot', 'Caligo', 'Cancer(Thyroid)', 'Celialgia', 'Chyluria', 'Contusion', 'Death', 'Digestive disease', 'Earring', 'Enuresis', 'Fatality', 'Flu', 'Gall Bladder', 'Gout', 'Heat rash', 'Herpes', 'Hypotherbia', 'Intestinal-Troubles', 'Larvicide(Veterinary)', 'Longevity', 'Mastitis', 'Molluscicide', 'Necklace', 'Odontalgic', 'Palate', 'Pertussis', 'Poison(Veterinary)', 'Preventitive(Disease)', 'Psoriasis', 'Reconstructive', 'Rheumatism(Veterinary)', 'Sclerosis(Abdomen)', 'Shortwindedness', 'Spiderlick', 'Stress', 'Tachycardia*(Veterinary)', 'Tinnitus?', 'Tumor(Feet)', 'Tumor(Viscera)', 'Vaginitis', 'Water vine', 'Tuberculosia', 'Narcotic(Veterinary)', 'Antidote(Laportea)', 'Dyspepsia(Veterinary)', 'Love-Charm', 'Sclerosis(Anus)', 'Weariness']


In [29]:
filter = duke.ACTIVITY.str.contains("Stomach")
filt_df = duke[filter]
val_stomach = filt_df.ACTIVITY.unique().tolist()

Adomen and Abdominal with "Abdom":

In [30]:
filter = duke.ACTIVITY.str.contains("Abdom")
filt_df = duke[filter]
val_abdom = filt_df.ACTIVITY.unique().tolist()

Intestine and Intestinal with "Intestin":

In [34]:
filter = duke.ACTIVITY.str.contains("Intestin")
filt_df = duke[filter]
val_intestin = filt_df.ACTIVITY.unique().tolist()

In [35]:
filter = duke.ACTIVITY.str.contains("Belly")
filt_df = duke[filter]
val_belly = filt_df.ACTIVITY.unique().tolist()

In [36]:
filter = duke.ACTIVITY.str.contains("Digest")
filt_df = duke[filter]
val_digest = filt_df.ACTIVITY.unique().tolist()

In [44]:
print(val_stomach)
print(val_abdom)
print(val_intestin)
print(val_belly)
print(val_digest)

['Ache(Stomach)', 'Ache(Stomach)(Veterinary)', 'Cancer(Stomach)', 'Polyp(Stomach)', 'Sclerosis(Stomach)', 'Stomachic', 'Stomach', 'Stomach(Veterinary)', 'Stomachic(Veterinary)', 'Tumor(Stomach)']
['Abdomen', 'Cancer(Abdomen)', 'Polyp(Abdomen)', 'Sclerosis(Abdomen)', 'Tumor(Abdomen)', 'Tumor(Abdomen)(Veterinary)']
['Cancer(Intestine)', 'Intestinal', 'Intestinal-Ailments', 'Intestinal-Catarrh', 'Intestinal tonic', 'Intestinal-Troubles', 'Intestine', 'Polyp(Intestine)', 'Tumor(Intestine)', 'Intestine(Veterinary)', 'Intestinal worms']
['Sclerosis(Belly)', 'Tumor(Belly)']
['Cancer(Digestive)', 'Digestion', 'Digestive', 'Digestive disease', 'Tumor(Digestive)']


# Data classification:

Activities - symptoms

More user friendly/human input

Provide a list of actual symptoms for query input

## Sourcing Symptoms:

In [38]:
gb_symptoms_df = pd.read_csv("../data/GOOGLE_SYMPTOMS/2022_country_weekly_2022_GB_weekly_symptoms_dataset.csv")
us_symptoms_df = pd.read_csv("../data/GOOGLE_SYMPTOMS/2022_country_weekly_2022_US_weekly_symptoms_dataset.csv")


In [39]:
gb_symptoms_df.head()

,country_region_code,country_region,sub_region_1,sub_region_1_code,sub_region_2,sub_region_2_code,place_id,date,symptom:Abdominal obesity,symptom:Abdominal pain,...,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
0,GB,United Kingdom,NaN,NaN,NaN,NaN,ChIJqZHHQhE7WgIReiWIMkOg-MQ,2022-01-03,3.28,7.08,...,1.83,0.34,0.67,4.32,0.51,4.33,0.65,0.21,0.63,0.47
1,GB,United Kingdom,NaN,NaN,NaN,NaN,ChIJqZHHQhE7WgIReiWIMkOg-MQ,2022-01-10,3.16,6.82,...,1.82,0.37,0.67,4.36,0.49,4.29,0.62,0.22,0.64,0.49
2,GB,United Kingdom,NaN,NaN,NaN,NaN,ChIJqZHHQhE7WgIReiWIMkOg-MQ,2022-01-17,3.13,7.02,...,1.85,0.38,0.65,4.33,0.51,4.33,0.62,0.21,0.70,0.45
3,GB,United Kingdom,NaN,NaN,NaN,NaN,ChIJqZHHQhE7WgIReiWIMkOg-MQ,2022-01-24,3.15,7.12,...,1.86,0.39,0.65,4.28,0.52,4.39,0.65,0.22,0.65,0.46
4,GB,United Kingdom,NaN,NaN,NaN,NaN,ChIJqZHHQhE7WgIReiWIMkOg-MQ,2022-01-31,3.17,7.07,...,1.91,0.39,0.66,4.21,0.49,4.22,0.66,0.22,0.67,0.46


In [40]:
us_symptoms_df.head()

,country_region_code,country_region,sub_region_1,sub_region_1_code,sub_region_2,sub_region_2_code,place_id,date,symptom:Abdominal obesity,symptom:Abdominal pain,...,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
0,US,United States,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2022-01-03,2.48,5.64,...,1.39,0.29,0.53,3.92,0.37,3.46,0.57,0.19,0.60,0.44
1,US,United States,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2022-01-10,2.41,5.47,...,1.36,0.30,0.62,3.85,0.37,3.53,0.57,0.20,0.62,0.53
2,US,United States,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2022-01-17,2.52,5.54,...,1.40,0.32,0.59,3.91,0.36,3.69,0.58,0.20,0.61,0.45
3,US,United States,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2022-01-24,2.48,5.50,...,1.40,0.33,0.55,3.93,0.36,3.80,0.59,0.20,0.62,0.46
4,US,United States,NaN,NaN,NaN,NaN,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,2022-01-31,2.50,5.48,...,1.40,0.34,0.53,3.90,0.35,3.59,0.56,0.20,0.60,0.44


Both have 430 columns.

In [41]:
df_columns = list(us_symptoms_df.columns)
print(df_columns[:15])

['country_region_code', 'country_region', 'sub_region_1', 'sub_region_1_code', 'sub_region_2', 'sub_region_2_code', 'place_id', 'date', 'symptom:Abdominal obesity', 'symptom:Abdominal pain', 'symptom:Acne', 'symptom:Actinic keratosis', 'symptom:Acute bronchitis', 'symptom:Adrenal crisis', 'symptom:Ageusia']


In [42]:
symptoms = list()

for col in df_columns:
    if "symptom:" not in col:
        continue
    col = col.replace("symptom:", "")
    col = col.capitalize()
    symptoms.append(col)

In [43]:
print(symptoms[::10])

['Abdominal obesity', 'Amblyopia', 'Anxiety', 'Astigmatism', "Beau's lines", 'Blushing', 'Bruise', 'Cataplexy', 'Clouding of consciousness', 'Constipation', 'Dementia', 'Dizziness', 'Ear pain', 'Erythema chronicum migrans', 'Fecal incontinence', 'Food intolerance', 'Goitre', 'Heart murmur', 'Hepatotoxicity', 'Hyperglycemia', 'Hypertrophy', 'Hypothyroidism', 'Ingrown hair', 'Kidney stone', 'Low-grade fever', 'Middle back pain', 'Muscle weakness', 'Neonatal jaundice', 'Obesity', 'Pain', 'Peripheral neuropathy', 'Podalgia', 'Pruritus ani', 'Rectal prolapse', 'Scar', 'Shortness of breath', 'Snoring', 'Stuttering', 'Tenderness', 'Trichoptilosis', 'Uterine contraction', 'Visual acuity', 'Hyperhidrosis']
